In [7]:
!pip install python-Levenshtein
!pip install textdistance
!pip install -U pandas
!pip install -U openpyxl

http://pixelbeat.jp/text-matching-3-approach-with-python/

https://www.y-shinno.com/python-difflib/

https://data-analysis-stats.jp/%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92/python%E3%81%A7%E3%81%AE%E6%96%87%E5%AD%97%E5%88%97%E3%81%AE%E8%B7%9D%E9%9B%A2%E3%82%84%E9%A1%9E%E4%BC%BC%E6%80%A7/


In [8]:
from google.colab import files
uploaded_files = files.upload()
uploaded_file_names = list(uploaded_files.keys())
uploaded_file_names.sort()
print(uploaded_file_names)

Saving 進撃辞書.xlsx to 進撃辞書 (2).xlsx
['進撃辞書.xlsx']


In [13]:
from difflib import SequenceMatcher
import itertools
import pandas as pd
from heapq import nlargest
from difflib import get_close_matches
import Levenshtein
import textdistance

df = pd.read_excel(uploaded_file_names[0],header=None)
lst = list(df[0])

keyword = input("文字を入力してください＞")


def get_lev_dist(str1,str2):
  # レーベンシュタイン距離の取得
  lev_dist = Levenshtein.distance(str1, str2)
  # 標準化(長い方の文字列の長さで割る)
  devider = len(str1) if len(str1) > len(str2) else len(str2)
  lev_dist = lev_dist / devider
  # 指標を合わせる(0:完全不一致 → 1:完全一致)
  lev_dist = 1 - lev_dist
  return lev_dist


#文字列同士の最小編集距離で表される距離
#diff.get_close_matchesからコピペ
def get_close_lev(word, possibilities, n=3, cutoff=0.6):
  if not n >  0:
    raise ValueError("n must be > 0: %r" % (n,))
  if not 0.0 <= cutoff <= 1.0:
      raise ValueError("cutoff must be in [0.0, 1.0]: %r" % (cutoff,))
 
  result = []
  for dic in lst:
    d = get_lev_dist(keyword, dic)
    if d >= cutoff:
      result.append((d, dic))

  # Move the best scorers to head of list
  result = nlargest(n, result)
  # Strip scores for the best n matches
  return  [x for score, x in result]



def get_close_jaro(word, possibilities, n=3, cutoff=0.6):
  if not n >  0:
    raise ValueError("n must be > 0: %r" % (n,))
  if not 0.0 <= cutoff <= 1.0:
      raise ValueError("cutoff must be in [0.0, 1.0]: %r" % (cutoff,))
 
  result = []
  for dic in lst:
    d = Levenshtein.jaro_winkler(keyword, dic)
    if d >= cutoff:
      result.append((d, dic))

  # Move the best scorers to head of list
  result = nlargest(n, result)
  # Strip scores for the best n matches
  return  [x for score, x in result]




def get_close_func(func,word, possibilities, n=3, cutoff=0.6):
  if not n >  0:
    raise ValueError("n must be > 0: %r" % (n,))
  if not 0.0 <= cutoff <= 1.0:
      raise ValueError("cutoff must be in [0.0, 1.0]: %r" % (cutoff,))
 
  result = []
  for dic in lst:
    d = func(keyword, dic)
    if d >= cutoff:
      result.append((d, dic))

  # Move the best scorers to head of list
  result = nlargest(n, result)
  # Strip scores for the best n matches
  return  [x for score, x in result]





#レーベンシュタイン距離法 文字列同士の最小編集距離で表される距離
ret = get_close_lev(keyword, lst, n=3, cutoff=0)
print(ret)

#ゲシュタルトパターンマッチング 文字列同士の連続する共通部分を抜き出し、抜き出した文字列の前後に対しても同じ処理を繰り返す
ret = get_close_matches(keyword, lst, n=3, cutoff=0)
print(ret)

#ジャロ・ウィンクラー距離法 文字列同士の一致する文字数と置換の要不要から距離を計算する
ret = get_close_jaro(keyword, lst, n=3, cutoff=0)
print(ret)


ret = get_close_func(textdistance.rle_ncd,keyword, lst, n=3, cutoff=0)
print(ret)

ret = get_close_func(textdistance.arith_ncd, keyword, lst, n=3, cutoff=0)
print(ret)


文字を入力してください＞吾輩は猫である
['勝負は今、ここで決める！', 'つまり、巨人の正体は、人間であると？', 'エレン、今は他にすべきことがある']
['つまり、巨人の正体は、人間であると？', '勝負は今、ここで決める！', 'エレン、今は他にすべきことがある']
['勝負は今、ここで決める！', 'いや、意味はある。エルヴィンがこいつらを選んだんだからな', '私は何でも協力する。マーレを皆殺しに出来るのなら、私は何だってやる！']
['３・４メートル級に苦戦し、数ヶ所で負傷者を出している。今の調査兵団には以前のような力は無い。だが、それだけの損害が無ければ、決してここまでたどり着けなかった', '黙ってちゃ分からないよ。普通はそんな１人でしゃべったりはしないと思うんだ？\u3000私は鏡に映る自分に、話しかけたりしたことが無いからさ', '騙された奴が負けたたったそれだけのことだ']
['いいや？ちっとも。', '頼む……静かに……', '待って下さい']
